In [1]:
import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
import folium
from ipynb.fs.defs.Local_search import get_row
from ipynb.fs.defs.Local_search import getName
from ipynb.fs.defs.functions import translate_name
from ipynb.fs.defs.functions import get_row
from ipynb.fs.defs.functions import get_column
from tqdm.notebook import tqdm as tqdm

df = pd.read_csv("../Dati/request_cr.csv", delimiter=",", encoding = "ISO-8859-1")
dataframe = pd.read_csv("../Dati/num_orders.csv", delimiter=";", encoding = "ISO-8859-1")
coor = pd.read_csv('../Dati/altro/CitiesLocation.csv', delimiter=";", encoding = "ISO-8859-1", index_col = 0)
dft = pd.read_csv("../Dati/distanze/corrected_times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1_MVLS = pd.read_csv("../Dati/turni_f/multi_veich_LS1.csv", delimiter=";", encoding = "ISO-8859-1")
fur2_MVLS = pd.read_csv("../Dati/turni_f/multi_veich_LS2.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_2opt = pd.read_csv("../Dati/turni_f/2-opt1_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur2_2opt = pd.read_csv("../Dati/turni_f/2-opt2_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_greedy = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur2_greedy = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";", encoding = "ISO-8859-1")

def manage_exception(string):
    if(string.lower().strip() == 'piadena drizzona'):
        return 'Piadena, Platina, 40'
    elif(string.lower().strip() == 'brescia'):
        return 'manerbio, insurrezione nazionale'
    elif(string.lower().strip() == 'centro fumetto a. pazienza' or string.lower().strip() == 'cremona. la piccola biblioteca' or string.lower().strip() == 'uff. bib. cremona'):
        return 'cremona, ugolani dati, 4'
    elif(string.lower().strip() == 'casale cremasco vidolasco'):
        return 'Casale Cremasco-Vidolasco, Roma, 56'
    elif(string.lower().strip() == 'cremona-m.i.t.o.'):
        return 'Cremona, Palosca n. 2'
    elif(string.lower().strip() == 'cremona - manin (scolastica)'):
        return 'Cremona, Felice Cavallotti, 2'
    elif(string.lower().strip() == 'gadesco pieve delmona'):
        return 'Gadesco-Pieve Delmona, Giovanni Lonati 9'
    elif(string.lower().strip() == 'cremona - cda'):
        return 'cremona, sesto 39'
    elif(string.lower().strip() == 'crema'):
        return 'Crema, Civerchi, 9'
    elif(string.lower().strip() == 'martignana di po, libertà, 64'):
        return 'martignana di po, libertà, 64'
    else:
        return string

In [54]:
loc = [coor.loc['Cremona, Ugolani Dati, 4']['Lat'], coor.loc['Cremona, Ugolani Dati, 4']['Lon']]
map_ = folium.Map(location = loc)
orig = ''
dest = ''


for i in range(len(dataframe)):
    try:
        orig = getName(coor, manage_exception(dataframe.iloc[i]['origin']))
        dest = getName(coor, manage_exception(dataframe.iloc[i]['destination']))
    except:
        continue
    point1 = [coor.loc[orig]['Lat'], coor.loc[orig]['Lon']]
    point2 = [coor.loc[dest]['Lat'], coor.loc[dest]['Lon']]
    points = [point1, point2]
    
    orders = dataframe.iloc[i]['num_orders']
    if(orders > 500):
        folium.PolyLine(points, color="black", weight=0.5, opacity=1).add_to(map_)
        #folium.PolyLine(points).add_to(map_)
    elif(orders >= 400 and orders < 500):
        folium.PolyLine(points, color="red", weight=0.5, opacity=1).add_to(map_)
    elif(orders >= 300 and orders < 400):
        folium.PolyLine(points, color="orange", weight=0.5, opacity=1).add_to(map_)
    elif(orders >= 200 and orders < 300):
        folium.PolyLine(points, color="darkblue", weight=0.5, opacity=1).add_to(map_)
    elif(orders >= 100 and orders < 200):
        folium.PolyLine(points, color="blue", weight=0.5, opacity=1).add_to(map_)
    elif(orders >= 0 and orders < 100):
        folium.PolyLine(points, color="lightblue", weight=0.5, opacity=1).add_to(map_)
    

C:\Users\Alberto\anaconda3\lib\site-packages\pandas\core\strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [13]:
#set(list(df['biblioteca_destinazione']))
origins = []
for x in set(list(df['biblioteca_partenza'])):
    try:
        origins.append(getName(dft, manage_exception(x)))
    except:
        continue
destins = []
for x in set(list(df['biblioteca_destinazione'])):
    try:
        destins.append(getName(dft, manage_exception(x)))
    except:
        continue
origins.sort()
destins.sort()

In [83]:
indx = [getName(dft, x) for x in dft.columns]
ord_numbers = pd.DataFrame({}, index = indx, columns = indx)
for i in range(len(dataframe)):
    try:
        orig = getName(dft, manage_exception(dataframe.iloc[i]['origin']))
        exc.append(dataframe.iloc[i]['origin'])
        dest = getName(dft, manage_exception(dataframe.iloc[i]['destination']))
        exc.append(dataframe.iloc[i]['destination'])
        orders = dataframe.iloc[i]['num_orders']
        ord_numbers.loc[orig,dest] = orders
    except:
        #print([orig, dest])
        continue  

In [84]:
ord_numbers=ord_numbers.replace(np.nan, 0)

In [85]:
ord_numbers.to_csv("../Dati/distanze/ord_numbers.csv", sep=";", encoding = "ISO-8859-1")

In [86]:
orders = pd.read_csv("../Dati/distanze/ord_numbers.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)